In [1]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Get list of Companies with clean data from allStock_5 year data

In [2]:
#List of company names in allStock data
df2 = pd.DataFrame()
df2 = pd.read_csv('C:/Users/archana.parihar/Downloads/ListOfCompanies.csv')
dfList = df2['Company'].tolist()
print('Number of companies in All stock data for which we have stock data: '+ str(len(dfList)))


Number of companies in All stock data for which we have stock data: 505


In [3]:
df1 = pd.DataFrame()
df1 = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks13-14-15.csv', parse_dates=True,index_col=0)


In [4]:
#Create dictionary with company name as index and count of observations as value
data = {}
for x in range(0,len(dfList)):
    data[dfList[x]] = []
for y in range(0,len(dfList)):
    data[dfList[y]] = (df1.Name == dfList[y]).sum()


In [5]:
#Find list of companies with insufficient data
count = 0
badCompanies = []
for i in data:
    if(data[i]<730):
        count = count+1
        badCompanies.append(i)
print('Companies with incomplete data : '+ str(count))
print(badCompanies)

Companies with incomplete data : 35
['BMY', 'DHR', 'ES', 'ICE', 'ORCL', 'O', 'IQV', 'COTY', 'FOXA', 'FOX', 'NWSA', 'NWS', 'ALLE', 'GOOG', 'NAVI', 'INFO', 'SYF', 'CFG', 'QRVO', 'WRK', 'KHC', 'PYPL', 'HPE', 'HPQ', 'CSRA', 'WLTW', 'UA', 'FTV', 'EVHC', 'HLT', 'DXC', 'BHGE', 'BHF', 'DWDP', 'APTV']


In [6]:
#Add those companies to bad company list, that are present in the Cluster from R and not there in allStocks data
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/ClustersFromR.csv')
ListOfCompFromClusters_R = df['Name'].tolist()
for i in range(0,len(dfList)):
    if dfList[i] not in ListOfCompFromClusters_R :
        badCompanies.append(dfList[i])
for i in range(0,len(ListOfCompFromClusters_R)):
    if ListOfCompFromClusters_R[i] not in dfList :
        badCompanies.append(ListOfCompFromClusters_R[i])
print('Number of companies in R clusters for which we company related data: '+ str(len(ListOfCompFromClusters_R)))
print('Companies with bad data : '+ str(len(badCompanies)))
print(badCompanies)

Number of companies in R clusters for which we company related data: 505
Companies with bad data : 55
['BMY', 'DHR', 'ES', 'ICE', 'ORCL', 'O', 'IQV', 'COTY', 'FOXA', 'FOX', 'NWSA', 'NWS', 'ALLE', 'GOOG', 'NAVI', 'INFO', 'SYF', 'CFG', 'QRVO', 'WRK', 'KHC', 'PYPL', 'HPE', 'HPQ', 'CSRA', 'WLTW', 'UA', 'FTV', 'EVHC', 'HLT', 'DXC', 'BHGE', 'BHF', 'DWDP', 'APTV', 'CBG', 'CHK', 'HCN', 'LUK', 'PCLN', 'PDCO', 'SIG', 'SNI', 'WYN', 'CSRA', 'ABMD', 'BKNG', 'CBRE', 'IPGP', 'JEF', 'MSCI', 'NKTR', 'SIVB', 'TTWO', 'WELL']


# Check Cointegration for 5 clusters - found by K means

In [7]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/ClustersFromR_KMeans.csv')
dfCluster1m = df.loc[df['Cluster'] == 1]
dfCluster2m = df.loc[df['Cluster'] == 2]
dfCluster3m = df.loc[df['Cluster'] == 3]
dfCluster4m = df.loc[df['Cluster'] == 4]
dfCluster5m = df.loc[df['Cluster'] == 5]

In [8]:
def test_CointegrationOfCombinedSeriesKMean(timeSeries):
    coint = False    
    #plot rolling statistics to visualize whether it is stationary or not
    ma = timeSeries.rolling(window=100).mean()
    mstd = timeSeries.rolling(window=100).std()
    
   # timeSeries.plot()
   # ma.plot(color='blue')
   # mstd.plot(color='black')
    
    #perform dickey fuller to check quantitatively
    dftest = adfuller(timeSeries,autolag= 'AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#lags used','No of observations used'])
    for key,value in dftest[4].items():
        dfoutput['Critical value (%s)'%key]= value

    #print(dfoutput) 
    #if dfoutput['Critical value (10%)']<-2.57  :
       # print('The pair is most likely Cointegrated')
       # coint = True
    #if dfoutput['Critical value (10%)']>-2.57  :
        #print('The pair is most likely NOT Cointegrated')
    if dfoutput['p-value']<=0.02:
        coint= True
        
    return coint

def ImproveStationarity_and_test_CointegrationKMean(dfX,dfY):
    #Convert closePrice of Time series X to log-Moving average time series for better value of regression coefficient
    dfX_logScale = np.log(dfX['close'])
    Xma = dfX_logScale.rolling(window=100).mean()
    logScaleMinusMovingAverage1 = dfX_logScale - Xma
    logScaleMinusMovingAverage1.dropna(inplace=True)

    #Convert closePrice of Time series Y to log-Moving average time series for better value of regression coefficient
    dfY_logScale = np.log(dfY['close'])
    Yma = dfY_logScale.rolling(window=100).mean()
    logScaleMinusMovingAverage2 = dfY_logScale - Yma
    logScaleMinusMovingAverage2.dropna(inplace=True)

    #Using linear regression to calculate the value of b for linearly combines Time series
    logScaleMinusMovingAverage1 = sm.add_constant(logScaleMinusMovingAverage1)
    results = sm.OLS(logScaleMinusMovingAverage2,logScaleMinusMovingAverage1['close']).fit()
    b = results.params['close']
    z = logScaleMinusMovingAverage1['close'] - (b * logScaleMinusMovingAverage2)
    coint = test_CointegrationOfCombinedSeriesKMean(z)
    return coint

In [9]:
def coIntTestForCluster(dfCluster,ClusterNumber):
    print('Cluster number: '+ str(ClusterNumber))
    ListOfCompInOneCluster = dfCluster['Name'].tolist()
    print('Old length of cluster : '+ str(len(ListOfCompInOneCluster)))
    for i in range(0,len(badCompanies)):
        if badCompanies[i] in ListOfCompInOneCluster :
            #print("Bad Company found in Cluster: " , badCompanies[i])
            ListOfCompInOneCluster.remove(badCompanies[i])
    print('New length of cluster : '+ str(len(ListOfCompInOneCluster)))

    ListOfPairs = list(itertools.combinations(ListOfCompInOneCluster, 2))
    #print(ListOfPairs[:2])
    #print(len(ListOfPairs))
    #print(ListOfPairs[0][0],ListOfPairs[0][1])

    pair_result_dict = {}
    for x in range(0,len(ListOfPairs)):
        pair_result_dict[ListOfPairs[x]] = []

    for i in range(0,len(ListOfPairs)):
        dfX = df1.loc[df1['Name']==ListOfPairs[i][0]]
        dfY = df1.loc[df1['Name']==ListOfPairs[i][1]]
        Result = ImproveStationarity_and_test_CointegrationKMean(dfX,dfY)
        pair_result_dict[ListOfPairs[i]] = Result

    ListOfValuesCluster_2=list(pair_result_dict.values())
    print('Total pairs:  ' + str(len(ListOfValuesCluster_2)))
    print('Number of True: ' + str(ListOfValuesCluster_2.count(True)))
    print('Number of False: ' + str(ListOfValuesCluster_2.count(False)))
    print('---------------------------------------------------------------')
    return pair_result_dict

listOfClusters = [dfCluster1m,dfCluster2m,dfCluster3m,dfCluster4m,dfCluster5m]
listOfResultDict = []

for c in range(0,len(listOfClusters)):
    res = coIntTestForCluster(listOfClusters[c],c)
    listOfResultDict.append(res)

Cluster number: 0
Old length of cluster : 16
New length of cluster : 14
Total pairs:  91
Number of True: 24
Number of False: 67
---------------------------------------------------------------
Cluster number: 1
Old length of cluster : 7
New length of cluster : 6
Total pairs:  15
Number of True: 1
Number of False: 14
---------------------------------------------------------------
Cluster number: 2
Old length of cluster : 480
New length of cluster : 439
Total pairs:  96141
Number of True: 20274
Number of False: 75867
---------------------------------------------------------------
Cluster number: 3
Old length of cluster : 1
New length of cluster : 1
Total pairs:  0
Number of True: 0
Number of False: 0
---------------------------------------------------------------
Cluster number: 4
Old length of cluster : 1
New length of cluster : 1
Total pairs:  0
Number of True: 0
Number of False: 0
---------------------------------------------------------------


In [10]:
ListOfCointPairs = []
for key, value in listOfResultDict[0].items():
    if value == True:
        ListOfCointPairs.append(key)
dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.head(5)
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_5_K_Means_cluster_1_Pvalue_02.csv',index=False, sep=',',encoding='utf-8')

print(len(ListOfCointPairs))

24


In [12]:
ListOfCointPairs = []
for key, value in listOfResultDict[1].items():
    if value == True:
        ListOfCointPairs.append(key)
dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.head(5)
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_5_K_Means_cluster_2_Pvalue_02.csv',index=False, sep=',',encoding='utf-8')
print(len(ListOfCointPairs))

1


In [11]:
ListOfCointPairs = []
for key, value in listOfResultDict[2].items():
    if value == True:
        ListOfCointPairs.append(key)
dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.head(5)
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_5_K_Means_cluster_3_Pvalue_02.csv',index=False, sep=',',encoding='utf-8')
print(len(ListOfCointPairs))

20274
